<a href="https://colab.research.google.com/github/imabansal/Assignment/blob/main/Llama_2_finetune_qlora_python_coder_GenAIPRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruction fine-tuning a Llama-2 model on generating Python code

## Installing and loading the libraries

In [1]:
#reference:
#https://github.com/edumunozsala/llama-2-7B-4bit-python-coder/blob/main/Llama-2-finetune-qlora-python-coder.ipynb
#https://towardsdatascience.com/fine-tune-your-own-llama-2-model-in-a-colab-notebook-df9823a04a32

In [2]:
!pip install --upgrade huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.8 MB/s eta 0:00:00


In [3]:
!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requiremen

In [4]:
!pip install ipywidgets==7.7.1
!pip install huggingface_hub
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.4 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset
from random import randrange

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM, PeftModel

from trl import SFTTrainer

## Setting Global Parameters

In [32]:
# The model that you want to train from the Hugging Face hub
model_id = "NousResearch/Llama-2-7b-hf"
# The instruction dataset to use
#dataset_name = "iamtarun/python_code_instructions_18k_alpaca"
dataset_name = "HuggingFaceH4/CodeAlpaca_20K"
# Dataset split
dataset_split= "train[:50%]"
# Fine-tuned model name
new_model = "llama-2-7b-int4-CodeAlpaca_50K"
# Huggingface repository
hf_model_repo="yashikagupta/llama-2-7b-int4-CodeAlpaca_20K_50percent"
# Load the entire model on the GPU 0
device_map = {"": 0}

################################################################################
# bitsandbytes parameters
################################################################################
# Activate 4-bit precision base model loading
use_4bit = True
# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"
# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"
# Activate nested quantization for 4-bit base models (double quantization)
use_double_nested_quant = False

################################################################################
# QLoRA parameters
################################################################################
# LoRA attention dimension
lora_r = 64
# Alpha parameter for LoRA scaling
lora_alpha = 16
# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# TrainingArguments parameters
################################################################################
# Output directory where the model predictions and checkpoints will be stored
output_dir = new_model
# Number of training epochs
num_train_epochs = 1
# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = False
# Batch size per GPU for training
per_device_train_batch_size = 1
# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1 # 2
# Enable gradient checkpointing
gradient_checkpointing = True
# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3
# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4 #1e-5
# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001
# Optimizer to use
optim = "paged_adamw_32bit"
# Learning rate schedule
lr_scheduler_type = "cosine" #"constant"
# Number of training steps (overrides num_train_epochs)
max_steps = -1
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = False
# Save checkpoint every X updates steps
save_steps = 0
# Log every X updates steps
logging_steps = 25
# Disable tqdm
disable_tqdm= True

################################################################################
# SFTTrainer parameters
################################################################################
# Maximum sequence length to use
max_seq_length = 2048 #None
# Pack multiple short examples in the same input sequence to increase efficiency
packing = True #False


## Connect to Huggingface Hub

You can log in to Hugging Face Hub interactively

In [7]:
from huggingface_hub import notebook_login
# Log in to HF Hub
notebook_login()

Or you can provide .env file containing the Hugging Face token

In [8]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

# Load the enviroment variables
load_dotenv()
# Login to the Hugging Face Hub
login(token=os.getenv("HF_HUB_TOKEN"))

## Load the dataset with the instruction set

In [9]:
# Load dataset from the hub
dataset = load_dataset(dataset_name, split=dataset_split)
# Show dataset size
print(f"dataset size: {len(dataset)}")
# Show an example
print(dataset[randrange(len(dataset))])


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/18019 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2003 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/HuggingFaceH4___parquet/HuggingFaceH4--CodeAlpaca_20K-d5a02b8586b0d4b1/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
dataset size: 9010
{'prompt': "Write Python code to merge two given dictionaries into a single dictionary.\nDictionary 1: {'A':1, 'B':2} \nDictionary 2: {'C':3, 'D':4}", 'completion': 'def merge_dicts(dic1, dic2): \n    res = {**dic1, **dic2} \n    return res'}


In [10]:
# Check the dataset structure
dataset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 9010
})

In [11]:
# Show a random example
print(dataset[randrange(len(dataset))])

{'prompt': 'You are given a string of words. Count how many words the string contains.\nwords = "This is a testing string"', 'completion': 'def word_count(words):\n    count = 0\n    for word in words.split():\n        count+=1\n    return count'}


To fine-tune our model, we need to convert our structured examples into a collection of tasks described via instructions. We define a formatting_function that takes a sample and returns a string with our instruction format.

In [12]:
# Set the instruction format for iamtarun/python_code_instructions_18k_alpaca
"""def format_instruction(sample):
	return f### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the following Task:

### Task:
{sample['instruction']}

### Input:
{sample['input']}

### Response:
{sample['output']}
"""

"def format_instruction(sample):\n\treturn f### Instruction:\nUse the Task below and the Input given to write the Response, which is a programming code that can solve the following Task:\n\n### Task:\n{sample['instruction']}\n\n### Input:\n{sample['input']}\n\n### Response:\n{sample['output']}\n"

In [13]:
# Set the instruction format for HuggingFaceH4/CodeAlpaca_20K
def format_instruction(sample):
	return f"""### Instruction:
You are a coding assistant that will write a Solution to resolve the following Task:"

### Task:
{sample['prompt']}

### Solution:
{sample['completion']}
"""

In [14]:
# Show a formatted instruction
print(format_instruction(dataset[randrange(len(dataset))]))


### Instruction:
You are a coding assistant that will write a Solution to resolve the following Task:"

### Task:
Convert a JSON object to corresponding HTML table format.
{ "name": "John", "age": 31, "city": "New York"}

### Solution:
<table>
     <tr>
        <td>Name</td>
        <td>John</td>
    </tr>
    <tr>
        <td>Age</td>
        <td>31</td>
    </tr>
    <tr>
        <td>City</td>
        <td>New York</td>
    </tr>
</table>



## Instruction fine-tune a Llama 2 model using trl and the SFTTrainer

We will use the recently introduced method in the paper "QLoRA: Quantization-aware Low-Rank Adapter Tuning for Language Generation" by Tim Dettmers et al. QLoRA is a new technique to reduce the memory footprint of large language models during finetuning, without sacrificing performance.

Quantize the pre-trained model to 4 bits and freeze it.
Attach small, trainable adapter layers. (LoRA)
Finetune only the adapter layers while using the frozen quantized model for context.

In [15]:
# Get the type
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_use_double_quant=use_double_nested_quant,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype
)

In [16]:
# Load the pretrained model
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache = False, device_map=device_map)
model.config.pretraining_tp = 1

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The SFTTrainer supports a native integration with peft, which makes it super easy to efficiently instruction tune LLMs. We only need to create our LoRAConfig and provide it to the trainer.

In [17]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        r=lora_r,
        bias="none",
        task_type="CAUSAL_LM",
)
# Not necessary when using SFTTrainer
# prepare model for training
# model = prepare_model_for_kbit_training(model)
# model = get_peft_model(model, peft_config)

Before we can start our training we need to define the hyperparameters (TrainingArguments) we want to use

In [18]:
# Define the training arguments
args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size, # 6 if use_flash_attention else 4,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=gradient_checkpointing,
    optim=optim,
    #save_steps=save_steps,
    logging_steps=logging_steps,
    save_strategy="epoch",
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    #bf16=bf16,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    #max_steps=max_steps,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    disable_tqdm=disable_tqdm,
    report_to="tensorboard",
    seed=42
)

We now have every building block we need to create our SFTTrainer to start then training our model.

In [19]:
# Create the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=packing,
    formatting_func=format_instruction,
    args=args,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Start training our model by calling the train() method on our Trainer instance.

In [20]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model in local
trainer.save_model()


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.1436, 'learning_rate': 1.845018450184502e-05, 'epoch': 0.0}
{'loss': 0.9767, 'learning_rate': 3.690036900369004e-05, 'epoch': 0.01}
{'loss': 0.8022, 'learning_rate': 5.535055350553506e-05, 'epoch': 0.01}
{'loss': 0.7099, 'learning_rate': 7.380073800738008e-05, 'epoch': 0.01}
{'loss': 0.6382, 'learning_rate': 9.22509225092251e-05, 'epoch': 0.01}
{'loss': 0.5983, 'learning_rate': 0.00011070110701107013, 'epoch': 0.02}
{'loss': 0.5668, 'learning_rate': 0.00012915129151291514, 'epoch': 0.02}
{'loss': 0.5747, 'learning_rate': 0.00014760147601476016, 'epoch': 0.02}
{'loss': 0.5655, 'learning_rate': 0.00016605166051660516, 'epoch': 0.02}
{'loss': 0.5739, 'learning_rate': 0.0001845018450184502, 'epoch': 0.03}
{'loss': 0.562, 'learning_rate': 0.0001999998966129416, 'epoch': 0.03}
{'loss': 0.5673, 'learning_rate': 0.000199994565766025, 'epoch': 0.03}
{'loss': 0.5585, 'learning_rate': 0.00019998115829707134, 'epoch': 0.04}
{'loss': 0.5742, 'learning_rate': 0.00019995967528901003, 'epoc

In [21]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [22]:
new_model

'llama-2-7b-int4-CodeAlpaca_50K'

## Merge the model and the adapters and save it

When running in a T4 instance we have to clean the memory

In [23]:
# Empty VRAM
del model
del trainer
import gc
gc.collect()
gc.collect()

19885

In [24]:
torch.cuda.empty_cache() # PyTorch thing

In [25]:
gc.collect()

0

Reload the trained and saved model and merge it then we can save the whole model

In [26]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)





Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


In [28]:
model = PeftModel.from_pretrained(base_model, new_model)
merged_model = model.merge_and_unload()

In [29]:
"""
# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")

"""

'\n# Save the merged model\nmerged_model.save_pretrained("merged_model",safe_serialization=True)\ntokenizer.save_pretrained("merged_model")\n\n'

In [33]:
# push merged model to the hub

!huggingface-cli login
merged_model.push_to_hub(hf_model_repo)
tokenizer.push_to_hub(hf_model_repo)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/yashikagupta/llama-2-7b-int4-CodeAlpaca_20K_50percent/commit/e30d75c2b9719e756010791f5c28dced23d77891', commit_message='Upload tokenizer', commit_description='', oid='e30d75c2b9719e756010791f5c28dced23d77891', pr_url=None, pr_revision=None, pr_num=None)